# Chi-Square Tests

There are a total of 2 different chi-square tests for categorical attributes.

1. Chi-square goodness of fit - Useful for 1 categorical attribute
2. Chi-square test of independence - Useful for 2 categorical attributes

Let's explore some data related to the [General Social Survey](https://gss.norc.org/).

In [ ]:
library(tidyverse)

head(gss_cat)

## Chi-square goodness of fit

Suppose we were interested in know if the proportion of people from the GSS data are similar to the political breakdown currently in the United States. This link gives the breakdown of the [current political affiliation in the United States](https://news.gallup.com/poll/15370/party-affiliation.aspx) according to a running Gallup poll. 

The chi-square goodness of fit explores the extent to which a categorical attribute follows a specified distribution. The chi-square goodness of fit test is often framed as a way to evaluate if the sample data is representative of the population. As such, the following hypotheses would be a way to frame this analysis. 

$$
H_{0}:\ All\ categories\ follow\ the\ population\ distribution
$$

or 

$$ 
H_{0}:\ All\ categories\ follow\ the\ specified\ percentages/proportions
$$

To frame this analysis in the context of the current data, let's look at the `partyid` attribute in the data. 

In [ ]:
count(gss_cat, partyid) %>%
  mutate(prop = n / sum(n))

For this example, I'm going to collapse categories and assign a few to NA to omit from the analysis. The following code does this task and then returns a similar table to that shown above.

In [ ]:
gss_cat <- gss_cat %>%
  mutate(partyid_collapse = fct_collapse(partyid,
    other = c("No answer", "Don't know", "Other party"),
    rep = c("Strong republican", "Not str republican"),
    ind = c("Ind,near rep", "Independent", "Ind,near dem"),
    dem = c("Not str democrat", "Strong democrat")
  ))

count(gss_cat, partyid_collapse) %>%
  mutate(prop = n / sum(n))

### Chi-Square GoF Mechanics

The chi-square goodness of fit compares the observed cell counts with the expected cell counts. More formally, the chi-square test statistic is as follows:

$$
\chi^2 = \sum \frac{( O - E ) ^ 2}{E}
$$

where $O$ is the observed cell counts and $E$ are the expected cell counts. The expected cell counts are defined as the sample size times the hypothesized proportions/percentages (this is not completely statistically accurate, however, in many social science situations, this should be sufficient). For example:

$$
E = p_{H_{0}} * N
$$

where $p_{H_{0}}$ is the hypothesized proportions from the null hypothesis. The $\chi^2$ statistic follows a chi-square distribution with $k - 1$ degrees of freedom, where $k$ is the number of categories. 

Using the table above, these can be computed from the data and assuming the following as population proportions/percentages from the Gallup poll: Rep = 26%, Ind = 44%, Dem = 26%, other = 4%.

In [ ]:
chi_tab <- count(gss_cat, partyid_collapse) %>%
  mutate(prop = n / sum(n), 
         prop_h0 = c(.04, .26, .44, .26),
         E = prop_h0 * sum(n))

chi_tab

The $\chi^2$ statistic can be computed manually.

In [ ]:
chi_tab %>%
  mutate(num = (n - E)^2,
         chi_cell = num / E) %>%
  summarise(chi_square = sum(chi_cell))

More readily, using the `chisq.test()` function in R is easier. This function takes one primary argument, the attribute to do the chi-square goodness of fit test on. Optionally, the specific proportions need to be passed as well, which is typically desired unless equal percentages/proportions are desired.

In [ ]:
xsq_got <- chisq.test(table(gss_cat$partyid_collapse), p = c(.04, .26, .44, .26))

xsq_got

### Explore Differences

It is often of interest to explore differences, particularly if the chi-square goodness of fit test has a small p-value. This would indicate that the counts likely do not follow the assumed distribution, but where are the differences found? Residuals can help with this. The residuals are the difference in the observed and expected values divided by the square root of the expected values. 

$$
\chi^2_{resid} = \frac{(O - E)}{\sqrt{E}}
$$

These can be extracted directly from the model object saved when running the chi-square test. 

In [ ]:
xsq_got$residuals

## Chi-square Test of Independence

The chi-square test of independence is similar to that of the goodness of fit test, except now instead of a single attribute of interest, there are now more than one categorical attribute to be explored. The test of independence explores if the observed attributes are independent from one another. That is, if the two categorical attributes are indpendent, this would assume that the two attributes are proportionally distributed across all categories. The form of the $\chi^2$ test is the same as the GoT test:

$$
\chi^2 = \sum \frac{( O - E ) ^ 2}{E}
$$

However, different from the goodness of fit test, the expected values are computed differently. The expected cell counts are now defined as:

$$
E = N * p_{r} * p_{c}
$$

where $p_{r}$ is the margin proportion for the rows, ignoring the columns (that is, marginal row proportion) and $p_{c}$ is the margin proportion for the columns, ignoring the rows. Finally, the test has degrees of freedom equal to $(r - 1)(c - 1)$.

### Data Example

To explore this example, let's see if the political party affiliation differs across years that the GSS data were collected. The data are collected over 14 years, collected every other year. 

In [ ]:
count(gss_cat, year)

Suppose we were interested in exploring if there was a difference in political affiliation before and after 2010. 

In [ ]:
gss_cat <- gss_cat %>%
  mutate(year_2 = ifelse(year < 2010, "2000 to 2008", "2010 to 2014"))

addmargins(table(gss_cat$year_2, gss_cat$partyid_collapse))

Giving this table, a single expected value could be computed manually. 

$$
E_{1,1} = 21483 * (14927 / 21483) * (548 / 21483) = 380.766
$$

In [ ]:
21483 * (14927 / 21483) * (548 / 21483)

These could be computed for subsequent cell expected values, but these can be extracted directly when fitting the chi-square using the `chisq.test()` function. 

In [ ]:
xsq_ind <- chisq.test(table(gss_cat$year_2, gss_cat$partyid_collapse))

xsq_ind$expected

In [ ]:
xsq_ind

In [ ]:
xsq_ind$residuals

## Effect Sizes for chi-square tests

Effect sizes for chi-square tests can be important, particularly with large sample sizes as the chi-square can be highly sensitive to large sample sizes. In particular, very small differences can be found with small samples sizes. 

For the goodness of fit test, $Cohens\ W$ can be estimated as an effect size measure. This is computed as:

$$
Cohens\ W = \sqrt{\frac{\chi^2}{N}}
$$

For the test of independence, Cramer's V can be used.

$$
V = \sqrt{\frac{\chi^2}{N * df^{*}}}
$$
where $df^{*}$ is the smallest of $r - 1$ or $c - 1$.

Cramers V ranges from 0 to 1, where values closer to 1 indicate more variation is explained (ie, the attributes are not independent).

Cohen's W is similar to Cramers V, but it is not limited to range between 0 and 1. 

In [ ]:
sqrt(693.4 / sum(chi_tab$n))

In [ ]:
sqrt(64.399 / 21483 * 1)

The DescTools package can be used for Cramer's V as well.

In [ ]:
library(DescTools)

CramerV(table(gss_cat$year_2, gss_cat$partyid_collapse))